In [129]:
#pyvizenv environment needed
#commented out libraries & APIs we aren't currently using - kp

import requests
# initial imports
import os
import pandas as pd
#import panel as pn
#import plotly.express as px
#import plotly.graph_objects as go
#import holoviews as hv
#from pathlib import Path
import datetime
from dotenv import load_dotenv
import json


load_dotenv()
rapid_api_key = os.getenv("RAPID_API_KEY")

In [105]:
#import hvplot.pandas
#import matplotlib.pyplot as plt

In [106]:
# Read the Mapbox API key
#load_dotenv()
#mapbox_token = os.getenv("MAPBOX")
#px.set_mapbox_access_token(mapbox_token)

In [107]:
#alpaca_token = os.getenv("ALPACA_API_KEY")

In [184]:
# outputs textbox to key in stock symbol and uses api to get historical stock data for that stock
# use code to loop through list of tickers and output daily close data for x amount of time
# updated ticker list to be in same format - kp
# changed order in loop to print ticker symbol at top - kp
# updated url to make api data in json format, changed return to dump with indents - kp

tickers = ["BTC-USD", "DJIA", "GLD", "DX-Y.NYB", "QQQ", "SPY", "WTI"]

def data_grab(ticker):
    """
    Takes in arg "ticker" as a str object,
    and returns daily stock information for that
    symbol/ticker.
    
    date/open/high/low/close/volume/adjclose
    """
    
    #Inital Lists
    
    closing_price_list = []
    closing_date_list = []
    closing_volume_list = []
    
    #API Information
    
    url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v3/get-historical-data"
    url += "?format=json"
    querystring = {"symbol":""}
    querystring['symbol']=ticker
    headers = {
        'x-rapidapi-host': "apidojo-yahoo-finance-v1.p.rapidapi.com",
        'x-rapidapi-key': rapid_api_key
        }
    response_data = requests.get(url, headers=headers, params=querystring)
    daily_data = response_data.json()
    daily_prices = daily_data['prices']
    
    # For Loop grabbing API Data and appending lists
    
    for daily_info in daily_prices:
        try:
            closing_date_list.append(datetime.datetime.fromtimestamp(daily_info['date']).strftime('%Y-%m-%d'))
            closing_price_list.append(daily_info['close'])
            closing_volume_list.append(daily_info['volume'])
        except:
            print(" ")
    
    # Clean Date List incase of duplicates (stock splits etc.)
    
    clean_date_list = list(dict.fromkeys(closing_date_list))
    
    # Creating a Pandas DataFrame 
    
    d = {'Dates':clean_date_list, "Closing Prices":closing_price_list, "Volume":closing_volume_list}
    df = pd.DataFrame(data=d)
    
    # Return/Display DataFrame
    
    return df

#return(print(json.dumps(daily_data, indent=4)))

In [159]:
##############
# TEST  AREA #
##############

#Initial Variables
closing_price_list = []
closing_date_list = []
closing_volume_list = []

#API Information
url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v3/get-historical-data"
url = url + "?format=json"
querystring = {"symbol":""}
querystring['symbol']="TSLA"
headers = {
    'x-rapidapi-host': "apidojo-yahoo-finance-v1.p.rapidapi.com",
    'x-rapidapi-key': rapid_api_key
    }

#Response Information
response_data = requests.get(url, headers=headers, params=querystring)
daily_data = response_data.json()
daily_prices = daily_data['prices']

#Loop to grab closing price
for daily_info in daily_prices:
    try:
        closing_date_list.append(datetime.datetime.fromtimestamp(daily_info['date']).strftime('%Y-%m-%d'))
        closing_price_list.append(daily_info['close'])
        closing_volume_list.append(daily_info['volume'])
    except:
        print("N/A")
        
clean_date_list = list(dict.fromkeys(closing_date_list))

d = {'Dates':clean_date_list, "Closing Prices":closing_price_list, "Volume":closing_volume_list}
df = pd.DataFrame(data=d)
        
df

N/A


,Dates,Closing Prices,Volume
0,2020-09-18,442.149994,85995000
1,2020-09-17,423.429993,76779200
2,2020-09-16,441.760010,72279300
3,2020-09-15,449.760010,97298200
4,2020-09-14,419.619995,83020600
5,2020-09-11,372.720001,60717500
6,2020-09-10,371.339996,84930600
7,2020-09-09,366.279999,79465800
8,2020-09-08,330.209991,115465700
9,2020-09-04,418.320007,110321900


In [ ]:
# NEXT STEPS
# create dataframes with api data